# Problem set 1

- 1.2 Linear programs
    - Exercises 2, 6 and 8
- 1.3 Integer programs
    - Exercises 

## Setup

We import the necessary libraries.

In [1]:
using Cbc, JuMP

## 1.2 Linear programs

### Exercise 2

*MUCOW (Milk Undertakings, C and O, Waterloo) owns a herd of Holstein cows and a herd of Jersey cows.
For each herd, the total number of litres produced each day, and milk-fat content (as a percentage) are as
follows:*

|  | Litres produced | Percent milk-fat |
| :-: | :-: | :-: |
| Holstein | 500 | 3.7
| Jersey | 250 | 4.9

*The fat is split off and blended again to create various products.
For each product, the volume, required milk-fat percentage, and profit are as follows.
In particular, the milk-fat percentage must exactly equal what is specified.*

|  | Skimmed milk | 2% | Whole milk | Table cream | Whipping cream |
| :-: | :-: | :-: | :-: | :-: | :-: |
| Volume (litres) | 2 | 2 | 2 | 0.6 | 0.3 |
| Milk-fat requirement | 0% | 2% | 4% | 15% | 45% |
| Profit ($) | 0.1 | 0.15 | 0.2 | 0.5 | 1.2 |

***(a)*** *Formulate as an LP the problem of deciding how many items of each type to produce, so that the total profit is maximized.
Don’t worry about fractional numbers of items.
Write your formulation in matrix notation.*

Consider $x_1, \cdots, x_5$ the number of skimmed milk, ..., whipping cream, respectively.
We want to maximize the profit:

$$\text{max } 0.1 x_1 + 0.15 x_2 + 0.2 x_3 + 0.5 x_4 + 1.2 x_5$$

If we split the milk and fat, we have $30.75 L$ of fat and $719.25 L$ of "pure" milk.
So we write the constraints for fat and milk separately:

$$0 x_1 + 0.04 x_2 + 0.08 x_3 + 0.09 x_4 + 0.135 x_5 \le 30.75$$
$$2 x_1 + 1.96 x_2 + 1.92 x_3 + 0.51 x_4 + 0.165 x_5 \le 719.25$$

That is, if we solve this linear problem, we have how many of each product we can produce with the possible milk and fat, in the correct proportions milk-fat.

Let's solve this:

In [2]:
m = Model(with_optimizer(Cbc.Optimizer, logLevel = 0))

# Number of each product
@variable(m, x[1:5] >= 0)

# Maximize the profit
@objective(m, Max, 0.1x[1] + 0.15x[2] + 0.2x[3] + 0.5x[4] + 1.2x[5])

# Constraint of fat
@constraint(m, 0.04x[2] + 0.08x[3] + 0.09x[4] + 0.135x[5] <= 30.75)
# Constraint of milk
@constraint(m, 2x[1] + 1.96x[2] + 1.92x[3] + 0.51x[4] + 0.165x[5] <= 719.25)

@show m

m = A JuMP Model
Maximization problem with:
Variables: 5
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 2 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 5 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: COIN Branch-and-Cut (Cbc)
Names registered in the model: x


A JuMP Model
Maximization problem with:
Variables: 5
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 2 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 5 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: COIN Branch-and-Cut (Cbc)
Names registered in the model: x

In [3]:
optimize!(m)

println("\nProfit: ", objective_value(m))
for i=1:5
    println("x $i qty: ", value(x[i]))
end


Profit: 307.4166666666666
x 1 qty: 340.8333333333333
x 2 qty: 0.0
x 3 qty: 0.0
x 4 qty: 0.0
x 5 qty: 227.77777777777774


So the solution is to use all fat for whipping cream and use the rest of the milk to produce skimmed milk.

***(b)*** *MUCOW is told of a regulation change: 'skimmed milk' can now contain anything up to 0.1\% milk-fat, but no more.
How does the formulation of the problem change? Note the resulting formulation should also be an LP.*

We could change the above approach to include a variable with the milk-fat percentage for skimmed milk, but the problem would not be linear.
Instead, we create another problem.
We try to maximize the same objective function, but we include variables $M_1, \cdots, M_5, f_1, \cdots, f_5$ for the "pure" milk and fat used by each kind of product.
We create the restrictions of milk and fat:

$$\sum_{i = 1}^5 M_i \le 719.25, \ \ \sum_{i = 1}^5 f_i \le 30.75$$

The milk-fat requirements must be satisfied:

$$0 \le \frac{f_1}{M_1 + f_1} \le 0.001$$
$$\frac{f_2}{M_2 + f_2} = 0.02, \ \ \frac{f_3}{M_3 + f_3} = 0.04, \ \ \frac{f_4}{M_4 + f_4} = 0.15, \ \ \frac{f_5}{M_5 + f_5} = 0.45$$

We also calculate the number of each product:

$$x_1 = \frac{M_1 + f_1}{2}, \ \ x_2 = \frac{M_2 + f_2}{2}, \ \ x_3 = \frac{M_3 + f_3}{2}, \ \ x_4 = \frac{M_4 + f_4}{0.6}, \ \ x_5 = \frac{M_5 + f_5}{0.3}$$

In [4]:
m = Model(with_optimizer(Cbc.Optimizer, logLevel = 0))

# Number of each product
@variable(m, x[1:5] >= 0)
# Amount of milk
@variable(m, M[1:5] >= 0)
# Amount of fat
@variable(m, f[1:5] >= 0)

# Maximize the profit
@objective(m, Max, 0.1x[1] + 0.15x[2] + 0.2x[3] + 0.5x[4] + 1.2x[5])

# Constraint of fat
@constraint(m, sum(f[1:5]) <= 30.75)
# Constraint of milk
@constraint(m, sum(M[1:5]) <= 719.25)
# Milk-fat requirements
@constraint(m, f[1] <= 0.001M[1] + 0.001f[1])
@constraint(m, f[2] == 0.02M[2] + 0.02f[2])
@constraint(m, f[3] == 0.04M[3] + 0.04f[3])
@constraint(m, f[4] == 0.15M[4] + 0.15f[4])
@constraint(m, f[5] == 0.45M[5] + 0.45f[5])
# Number of each product
@constraint(m, 2x[1] == M[1] + f[1])
@constraint(m, 2x[2] == M[2] + f[2])
@constraint(m, 2x[3] == M[3] + f[3])
@constraint(m, 0.6x[4] == M[4] + f[4])
@constraint(m, 0.3x[5] == M[5] + f[5])

@show m

m = A JuMP Model
Maximization problem with:
Variables: 15
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 9 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 3 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 15 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: COIN Branch-and-Cut (Cbc)
Names registered in the model: M, f, x


A JuMP Model
Maximization problem with:
Variables: 15
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 9 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 3 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 15 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: COIN Branch-and-Cut (Cbc)
Names registered in the model: M, f, x

In [5]:
optimize!(m)

println("\nProfit: ", objective_value(m))
for i=1:5
    println("x $i qty: ", value(x[i]))
end
println("Milk-fat 1: ", value(f[1])/(value(f[1]) + value(M[1])))


Profit: 307.41666666666663
x 1 qty: 340.8333333333333
x 2 qty: 0.0
x 3 qty: 0.0
x 4 qty: 0.0
x 5 qty: 227.77777777777777
Milk-fat 1: 0.0


The solution to the last exercise was to use all fat for whipping cream and use the rest of the milk to produce skimmed milk.
The same here!

With respect to the last exercise, we allowed the sollution to give some fat to skimmed milk.
For each whipping cream, it is necessary $0.135 L$ of fat to gain $\$1.2$.
It is not worth it to use $0.1 L$ of fat to gain $\$0.1$ with skimmed milk.

## Exercise 6

We are given an $m$ by $n$ matrix $A$ and a vector $b$ in $\mathbb{R}^m$, for which the system $Ax = b$ has no solution.
Here is an example:

$$\begin{cases}
    2x_1 - x_2 = -1 \\
    x_1 + x_2 = 1 \\
    x_1 + 3x_2 = 4 \\
    -2x_1 + 4x_2 = 3
\end{cases}$$

We are interested in finding a vector $x \in \mathbb{R}^n$ that "comes close" to solving the system.
Namely, we want to find an $x \in \mathbb{R}^n$ whose *deviation* is mininum, and where the deviation of $x$ is defined to be

$$\sum_{i=1}^n\left|b_i - \sum_{j=1}^na_{ij}x_j\right|$$

(For the example system above, the vector $x = (1, 1)^\intercal$ has deviation $2 + 1 + 0 + 1 = 4$.)

***(a)*** *Show that a solution to the optimization problem*

$$\begin{align}
    &\text{minimize } \sum_{i=1}^{m} y_i \\
    &\text{subject to} \\
    &\hspace{2cm}\left|\sum_{j=1}^n a_{ij} x_j - b_i\right| \le y_i, \ \ \ \ i = 1, \cdots, m
\end{align}$$

*will give a vector $x$ of minimum deviation.*

Note that 

$$\text{minimize} \sum_{i=1}^n\left|b_i - \sum_{j=1}^na_{ij}x_j\right|$$

is the same as

$$\begin{align}
    &\text{minimize } \sum_{i=1}^{m} y_i \\
    &\text{subject to} \\
    &\hspace{2cm}\left|\sum_{j=1}^n a_{ij} x_j - b_i\right| = y_i, \ \ \ \ i = 1, \cdots, m
\end{align}$$

which has the same optimal solution (when it exists) as

$$\begin{align}
    &\text{minimize } \sum_{i=1}^{m} y_i \\
    &\text{subject to} \\
    &\hspace{2cm}\left|\sum_{j=1}^n a_{ij} x_j - b_i\right| \le y_i, \ \ \ \ i = 1, \cdots, m
\end{align}$$

***(b)*** *The problem of part **(a)** is not a LP.
(Why?)
Show that it can be formulated as an LP.*

The problem of **(a)** is not LP because of the absolute value.
But it is simple to make it LP:

$$\begin{align}
&\text{minimize } \sum_{i=1}^{m} y_i \\
&\text{subject to} \\
&\hspace{2cm}\sum_{j=1}^n a_{ij} x_j - b_i \le y_i, \ \ \ \ i = 1, \cdots, m \\
&\hspace{2cm}-y_i \le \sum_{j=1}^n a_{ij} x_j - b_i, \ \ \ \ i = 1, \cdots, m
\end{align}$$

***(c)*** *Suppose that we had instead defined the deviation of $x$ as*

$$\max_{1 \le i \le m} \left| b_i - \sum_{j = 1}^n a_{ij} x_j \right|$$

*(According to this definition, in the example above $x = (1, 1)^\intercal$ would have deviation $\max\{2, 1, 0, 1\} = 2$.)
With this new definition, write the problem of finding a vector of minimum deviation as an optimization problem, and show that this problem can also be formulated as an LP.*

The problem can be formulated as

$$\text{minimize} \max_{1 \le i \le m} \left| b_i - \sum_{j = 1}^n a_{ij} x_j \right|$$

When can do a trick in order to make it LP:

$$\begin{align}
    &\text{minimize } y \\
    &\text{subject to  } \\
    &\hspace{2cm} \left| b_i - \sum_{j = 1}^n a_{ij} x_j \right| \le y, \ \ \ \ i = 1, \cdots, m
\end{align}$$

## Reference

```bibtex
@book{guenin2014gentle,
    title={A Gentle Introduction to Optimization},
    author={Guenin, B. and K{\"o}nemann, J. and Tun{\c{c}}el, L.},
    isbn={9781107053441},
    lccn={2014008067},
    url={https://books.google.com.br/books?id=JC4DBAAAQBAJ},
    year={2014},
    publisher={Cambridge University Press}
}
```